In [1]:
%matplotlib inline
import matplotlib.pyplot as plt
import nengolib
import numpy as np
import pandas as pd
import random
import scipy as sp
from sklearn.metrics import accuracy_score
from sklearn.decomposition import PCA
from itertools import islice

import pytry

In [2]:
class PatternInterpolationTrial(pytry.Trial):
    def params(self):
        self.param('number of dimensions', n_dims=2),
        self.param('length of pattern', len_train=10),
        self.param('seed', p_seed=1),
        
    def evaluate(self, param): #function to fill in missing data points by interpolating
        import nengo
        data = pd.read_csv("multidata.csv", low_memory=False)
        
        x = np.array(data.iloc[:,11:195]).astype(float) #array of data for purple child (points in space for each frame)
        labs = np.array(data.iloc[:,443]).astype(str) #array of labels (purple child annotations, engagement) 218/443
        a = []
        for i in range(x.shape[1]):
            y = pd.Series(x[:,i])
            if i == 180:               # add these three lines
                y[y>0]-=np.pi*2   # add these three lines
                y += np.pi            # add these three lines
            z = y.interpolate(limit_direction='both')
            a.append(z)
        a = pd.DataFrame(a)
        a = a.dropna()
        a = np.array(a).T
        
        dt = 0.001
        target_dt = 1.0/30
        D = param.n_dims
        classify_score = {}
        accuracy = {}
        #seed = #random value seed
        
        def extract_pattern(start, end, target_dt): #function to extract consecutive frames with the same label 
            pattern = np.array(a[start:end,:]).astype(float)
            frames = np.array(data.iloc[start:end,9]).astype(int)

            good_indices = frames != -1
            frames = frames[good_indices]
            pattern = pattern[good_indices]

            fps = 30.0
            t_sample = (frames - frames[0])/fps

            t = np.arange(int(t_sample[-1]/target_dt))*target_dt

            result = []
            for i in range(pattern.shape[1]):       
                p = np.interp(t, t_sample, pattern[:,i])
                result.append(p)
            result = np.array(result).T

            return t, result

        start=[]
        start.append(0)
        end=[]
        label = []
        for i in range(1, (len(labs)-1)):
            if labs[i]!=labs[i-1]:
                start.append(i)
            if labs[i]!=labs[i+1]:
                end.append(i)
                label.append(labs[i])

        t_noplay=[]
        p_noplay=[]
        t_goal=[]
        p_goal=[]
        t_aim=[]
        p_aim=[]
        
        try:
            for i in range(1,(len(start)-1)):
                if label[i]==('noplay'):
                    ti, pi = extract_pattern(start[i], end[i], target_dt=target_dt)
                    t_noplay.append(ti)
                    p_noplay.append(pi)
                if label[i]==('goaloriented'):
                    ti, pi = extract_pattern(start[i], end[i], target_dt=target_dt)
                    t_goal.append(ti)
                    p_goal.append(pi)
                if label[i]==('aimless'):
                    ti, pi = extract_pattern(start[i], end[i], target_dt=target_dt)
                    t_aim.append(ti)
                    p_aim.append(pi)
        except IndexError:
            print('empty pattern')
        
############ randomly split data 80/20 into training and testing sets ############ 
        p_goal = random.sample(p_goal, len(p_goal))
        goal_train = p_goal[:(int(len(p_goal)*0.8))] 
        goal_test = p_goal[(int(len(p_goal)*0.8)):]

        p_noplay = random.sample(p_noplay, len(p_noplay))
        noplay_train = p_noplay[:(int(len(p_noplay)*0.8))]
        noplay_test = p_noplay[(int(len(p_noplay)*0.8)):]

        train_all = np.vstack(goal_train+noplay_train)
        pca_model = PCA(n_components=D).fit(train_all)

        goal_train_pca = np.vstack([pca_model.transform(p) for p in goal_train])
        noplay_train_pca = np.vstack([pca_model.transform(p) for p in noplay_train])
        goal_test_pca = np.vstack([pca_model.transform(p) for p in goal_test])
        noplay_test_pca = np.vstack([pca_model.transform(p) for p in noplay_test])

        T_train = param.len_train   # number of seconds to train on for each class
        T_test = 10    # number of seconds to test on for each class

        N_frames = int(T_train*30)
        training = np.vstack([goal_train_pca[:N_frames], noplay_train_pca[:N_frames]])
        assert len(training) == N_frames*2

        N_frames = int(T_test*30)
        testing = np.vstack([goal_test_pca[:N_frames], noplay_test_pca[:N_frames]])
        assert len(testing) == N_frames*2
        
############ TRAINING WITH 80% ############ 

        theta = 0.5
        net = nengo.Network(seed=param.seed)
        with net:
            rw = []
            for i in range(D):
                process = nengo.processes.PresentInput(np.hstack([goal_train_pca[:,i], noplay_train_pca[:,i]]), 
                                                           presentation_time=1.0/30)
                rw.append(nengolib.networks.RollingWindow(theta=theta, n_neurons=3000, 
                                                          process=process, 
                                                          neuron_type=nengo.LIFRate()))


            node_pool = nengo.Node(None, size_in=rw[0].state.size_out*D)

            start = 0
            for r in rw:
                nengo.Connection(r.state, node_pool[start:start+r.state.size_out])
                start += r.state.size_out



            stim = nengo.Node(nengo.processes.PresentInput(training, presentation_time=1.0/30))
            assert stim.size_out == D
            for i in range(D):
                nengo.Connection(stim[i], rw[i].input, synapse=None)

            p_node_pool = nengo.Probe(node_pool)


        sim = nengo.Simulator(net)
        with sim:
            sim.run(T_train*2) 
        
        pool_model = nengo.Network()
        with pool_model:
            pool = nengo.Ensemble(n_neurons=3000, dimensions=node_pool.size_out,
                                  neuron_type=nengo.LIFRate(), seed=p_seed)
        pool_sim = nengo.Simulator(pool_model)

        import nengo.utils.ensemble

        _, a = nengo.utils.ensemble.tuning_curves(pool, pool_sim, inputs=sim.data[p_node_pool])


        N = int(T_train*1000)
        target = np.hstack([np.ones(N), -np.ones(N)]).reshape(-1, 1)
        dec, info = nengo.solvers.LstsqL2(reg=0.1)(a, target)
        
        v = np.dot(a, dec)
        classify1 = np.isclose(v[:N], 1, atol=0.5)
        classify2 = np.isclose(v[N:], -1, atol=0.5)
        classify = np.append(classify1, classify2)
        score_train = np.mean(classify)
        
        classify_score[0]=score_train
        
        accuracy = dict(islice(enumerate(v), None, None, 100)) 
        accuracy_train={}
        for i in range(len(accuracy)):
            accuracy_train[i]=accuracy[i*100]

        for j in range(len(accuracy_train)):
            key_j = 'accuracy_train{}'.format(j)  
            accuracy_train[key_j] = accuracy_train.pop(j)
            
        #train_pattern = dict(islice(enumerate(training), None, None, 100))
        #training_pattern = {}
        #for i in range(len(train_pattern)):
            #training_pattern[i]=train_pattern[i*100]
        
        #for j in range(len(training_pattern)):
            #key_j = 'training_pattern{}'.format(j)  
            #training_pattern[key_j] = training_pattern.pop(j)

############ TESTING WITH 20% ############ 

        D = param.n_dims

        theta = 0.5
        test_net = nengo.Network(seed=param.seed)
        with test_net:
            rw = []
            for i in range(D):
                process = nengo.processes.PresentInput(np.hstack([goal_train_pca[:,i], noplay_train_pca[:,i]]), 
                                                           presentation_time=1.0/30)
                rw.append(nengolib.networks.RollingWindow(theta=theta, n_neurons=3000, 
                                                          process=process, 
                                                          neuron_type=nengo.LIFRate()))


            pool = nengo.Ensemble(n_neurons=3000, dimensions=node_pool.size_out,
                                  neuron_type=nengo.LIFRate(), seed=p_seed)

            start = 0
            for r in rw:
                nengo.Connection(r.state, pool[start:start+r.state.size_out])
                start += r.state.size_out



            stim = nengo.Node(nengo.processes.PresentInput(testing, presentation_time=1.0/30))
            assert stim.size_out == D
            for i in range(D):
                nengo.Connection(stim[i], rw[i].input, synapse=None)

            p_stim = nengo.Probe(stim)

            result = nengo.Node(None, size_in=1)
            nengo.Connection(pool.neurons, result, transform=dec.T, synapse=None)

            p_test_result = nengo.Probe(result)
            p_test_result_sample = nengo.Probe(result, sample_every = 0.1)


        test_sim = nengo.Simulator(test_net)
        test_sim.run(T_test*2) 
        
        N = int(T_test*1000)

        v = test_sim.data[p_test_result]
        classify1 = np.isclose(v[:N], 1, atol=0.5)
        classify2 = np.isclose(v[N:], -1, atol=0.5)
        classify = np.append(classify1, classify2)
        score_test = np.mean(classify)
        
        classify_score[1]=score_test
            
        accuracy = dict(islice(enumerate(v), None, None, 100)) 
        accuracy_test={}
        for i in range(len(accuracy)):
            accuracy_test[i]=accuracy[i*100]

        for j in range(len(accuracy_test)):
            key_j = 'accuracy_test{}'.format(j)  
            accuracy_test[key_j] = accuracy_test.pop(j)
        
        #test_pattern = dict(islice(enumerate(testing), None, None, 100))
        #testing_pattern = {}
        #for i in range(len(test_pattern)):
            #testing_pattern[i]=test_pattern[i*100]
        
        #for j in range(len(testing_pattern)):
            #key_j = 'testing_pattern{}'.format(j)  
            #testing_pattern[key_j] = testing_pattern.pop(j)

############ TESTING WITH AIMLESS PATTERNS ############ 

        p_aim = random.sample(p_aim, len(p_aim))
    
        aim_train_pca = np.vstack([pca_model.transform(p) for p in p_aim])

        N_frames = int(T_test*30)

        test_aim = np.vstack([aim_train_pca[:N_frames]])

        assert len(test_aim) == N_frames
        
        
        theta = 0.5
        aim_test_net = nengo.Network(seed=param.seed)
        with aim_test_net:
            rw = []
            for i in range(D):
                process = nengo.processes.PresentInput(np.hstack([goal_train_pca[:,i], noplay_train_pca[:,i]]), 
                                                           presentation_time=1.0/30)
                rw.append(nengolib.networks.RollingWindow(theta=theta, n_neurons=3000, 
                                                          process=process, 
                                                          neuron_type=nengo.LIFRate()))


            pool = nengo.Ensemble(n_neurons=3000, dimensions=node_pool.size_out,
                                  neuron_type=nengo.LIFRate(), seed=p_seed)

            start = 0
            for r in rw:
                nengo.Connection(r.state, pool[start:start+r.state.size_out])
                start += r.state.size_out



            stim = nengo.Node(nengo.processes.PresentInput(test_aim, presentation_time=1.0/30))
            assert stim.size_out == D
            for i in range(D):
                nengo.Connection(stim[i], rw[i].input, synapse=None)

            p_stim = nengo.Probe(stim)

            result = nengo.Node(None, size_in=1)
            nengo.Connection(pool.neurons, result, transform=dec.T, synapse=None)
            
            p_aim_result = nengo.Probe(result)
            p_aim_result_sample = nengo.Probe(result, sample_every=0.1)


        aim_test_sim = nengo.Simulator(aim_test_net)
        aim_test_sim.run(T_test)
        
        v = aim_test_sim.data[p_aim_result]
        classify = np.isclose(v, 0, atol=0.5)
        score_aim = np.mean(classify)
            
        classify_score[2]=score_aim
        
        #plt.plot(aim_test_sim.trange(), classify)
        #plt.plot(aim_test_sim.trange(), v)
        #plt.title('testing classification accuracy: %1.2f%%' % (score_aim*100))
        
        accuracy = dict(islice(enumerate(v), None, None, 100)) 
        accuracy_aim={}
        for i in range(len(accuracy)):
            accuracy_aim[i]=accuracy[i*100]

        for j in range(len(accuracy_aim)):
            key_j = 'accuracy_aim{}'.format(j)  
            accuracy_aim[key_j] = accuracy_aim.pop(j)
            
        #aim_pattern = dict(islice(enumerate(test_aim), None, None, 100))
        #aimless_pattern = {}
        #for i in range(len(aim_pattern)):
            #aimless_pattern[i]=aim_pattern[i*100]
        
        #for j in range(len(aimless_pattern)):
            #key_j = 'aimless_pattern{}'.format(j)  
            #aimless_pattern[key_j] = aimless_pattern.pop(j)

############ SAVE DATA ############ 
        classify_score['classify_train'] = classify_score.pop(0)
        classify_score['classify_test'] = classify_score.pop(1)
        classify_score['classify_aim'] = classify_score.pop(2)
    
        #for j in range(len(classify_score)):
        #    key_j = 'classify_{}'.format(j)  
        #    classify_score[key_j] = classify_score.pop(j)
        
        accuracy = {**accuracy_train, **accuracy_test, **accuracy_aim}
        
        #input_pattern = {**training_pattern, **testing_pattern, **aimless_pattern}
    
        results = {**classify_score, **accuracy}
        return results


In [3]:
for seed in range(20):
    PatternInterpolationTrial().run(seed=seed, p_seed=seed, len_train=140, data_dir='len_80_plus')

running PatternInterpolationTrial#20190427-084853-2e408369
empty pattern


C:\Users\mbartlett2\AppData\Local\Continuum\anaconda3\lib\site-packages\scipy\signal\filter_design.py:1551: BadCoefficients: Badly conditioned filter coefficients (numerator): the results may be meaningless
  "results may be meaningless", BadCoefficients)
C:\Users\mbartlett2\AppData\Local\Continuum\anaconda3\lib\site-packages\nengo\utils\numpy.py:79: FutureWarning: Using a non-tuple sequence for multidimensional indexing is deprecated; use `arr[tuple(seq)]` instead of `arr[seq]`. In the future this will be interpreted as an array index, `arr[np.array(seq)]`, which will result either in an error or a different result.
  v = a[inds]
C:\Users\mbartlett2\AppData\Local\Continuum\anaconda3\lib\site-packages\nengolib\signal\system.py:717: UserWarning: Filtering with non-SISO systems is an experimental feature that may not behave as expected.
  "expected.", UserWarning)


HtmlProgressBar cannot be displayed. Please use the TerminalProgressBar. It can be enabled with `nengo.rc.set('progress', 'progress_bar', 'nengo.utils.progress.TerminalProgressBar')`.

<IPython.core.display.Javascript object>

HtmlProgressBar cannot be displayed. Please use the TerminalProgressBar. It can be enabled with `nengo.rc.set('progress', 'progress_bar', 'nengo.utils.progress.TerminalProgressBar')`.

...

HtmlProgressBar cannot be displayed. Please use the TerminalProgressBar. It can be enabled with `nengo.rc.set('progress', 'progress_bar', 'nengo.utils.progress.TerminalProgressBar')`.

<IPython.core.display.Javascript object>

HtmlProgressBar cannot be displayed. Please use the TerminalProgressBar. It can be enabled with `nengo.rc.set('progress', 'progress_bar', 'nengo.utils.progress.TerminalProgressBar')`.

<IPython.core.display.Javascript object>

seed = 19
n_dims = 2
len_train = 140
p_seed = 19
accuracy_aim0 = array([-0.92916315])
accuracy_aim1 = array([-0.30668975])
accuracy_aim10 = array([0.29776358])
accuracy_aim11 = array([0.33719286])
accuracy_aim12 = array([0.41852478])
accuracy_aim13 = array([0.50156476])
accuracy_aim14 = array([0.53131938])
accuracy_aim15 = array([0.52290088])
accuracy_aim16 = array([0.48188594])
accuracy_aim17 = array([0.43721957])
accuracy_aim18 = array([0.38204126])
accuracy_aim19 = array([0.35199046])
accuracy_aim2 = array([-0.02284355])
accuracy_aim20 = array([0.35052407])
accuracy_aim21 = array([0.40317741])
accuracy_aim22 = array([0.4680326])
accuracy_aim23 = array([0.49983946])
accuracy_aim24 = array([0.50932088])
accuracy_aim25 = array([0.4737418])
accuracy_aim26 = array([0.42671817])
accuracy_aim27 = array([0.39057212])
accuracy_aim28 = array([0.36980099])
accuracy_aim29 = array([0.38144223])
accuracy_aim3 = array([0.26536943])
accuracy_aim30 = array([0.42688996])
accuracy_aim31 = array([0.464

In [ ]:
for seed in range(20):
    for dims in (1,2,3,4):
        PatternInterpolationTrial().run(seed=seed, p_seed=seed, n_dims=dims, data_dir='02-child-robot')

In [ ]:
for seed in range(20,100):
    for lens in (10,20,30,40):
        PatternInterpolationTrial().run(seed=seed, len_train=lens, data_dir='ICCM_lens')

In [ ]:
for seed in range(20,100):
    for lens in (10,20,30,40):
        for dims in (1,2,3,4):
            PatternInterpolationTrial().run(seed=seed, len_train=lens, n_dims=dims, data_dir='grid_search')

In [ ]:
PatternInterpolationTrial().run(len_train=5, n_dims=4, data_dir='len_5')

In [ ]:
PatternInterpolationTrial().run(len_train=5, n_dims=8, data_dir='len_5_dim_8')

In [ ]:
PatternInterpolationTrial().run(n_dims=3, data_dir='debug')

In [ ]:
PatternInterpolationTrial().run(len_train=10, data_dir='quick')

In [ ]:
PatternInterpolationTrial().run(len_train=80, n_dims=2, data_dir='80_secs')